# ChiselDB：调试友好的结构化数据库

我们推出了 ChiselDB，一个易于调试的结构化数据库。它将在硬件模块接口之间插入探针，并直接使用 Chisel 代码中的 DPI-C 来传输捆绑信息和数据。在错误分析方面，它支持 SQL 查询，因此比 Waveform 更易于使用。

要使用 ChiselDB，第一步是创建表。

下面是一个例子，调用这个 API 函数，可以创建一个 chiselDB 表并定义名称和数据类型。

In [ ]:
// Design source code:  XiangShan/utility/src/main/scala/utility/ChiselDB.scala
// Usage:  Create table

// API: def createTable[T <: Record](tableName: String, hw: T): Table[T] 
import huancun.utils.ChiselDB

class MyBundle extends Bundle { 
    val fieldA = UInt(10.W)
    val fieldB = UInt(20.W) 
} 
val table = ChiselDB.createTable("MyTableName", new MyBundle)

第二步是注册探测器。

您可以使用函数 table.log 来记录 en 为真时的信号值。

In [ ]:
// Usage:  Register probes

/* APIs
def log(data: T, en: Bool, site: String = "", clock: Clock, reset: Reset)
def log(data: Valid[T], site: String, clock: Clock, reset: Reset): Unit
def log(data: DecoupledIO[T], site: String, clock: Clock, reset: Reset): Unit
*/
table.log(
  data = my_data /* hardware of type T */,
  en = my_cond,   site = "MyCallSite",
  clock = clock,   reset = reset
)

这是一个完整的例子，这里记录一下 L2TLB 的 missqueue 的输入输出请求。

// Example:  XiangShan/src/main/scala/xiangshan/cache/mmu/L2TLB.scala

class L2TlbMissQueueDB(implicit p: Parameters) extends TlbBundle {
   val vpn = UInt(vpnLen.W)
}

val L2TlbMissQueueInDB, L2TlbMissQueueOutDB = Wire(new L2TlbMissQueueDB)
L2TlbMissQueueInDB.vpn := missQueue.io.in.bits.vpn
L2TlbMissQueueOutDB.vpn := missQueue.io.out.bits.vpn

val L2TlbMissQueueTable = ChiselDB.createTable(
    "L2TlbMissQueue_hart" + p(XSCoreParamsKey).HartId.toString, new L2TlbMissQueueDB)

L2TlbMissQueueTable.log(L2TlbMissQueueInDB, missQueue.io.in.fire, "L2TlbMissQueueIn", clock, reset)
L2TlbMissQueueTable.log(L2TlbMissQueueOutDB, missQueue.io.out.fire, "L2TlbMissQueueOut", clock, reset)

In [ ]:
// Example:  XiangShan/src/main/scala/xiangshan/cache/mmu/L2TLB.scala

class L2TlbMissQueueDB(implicit p: Parameters) extends TlbBundle {
   val vpn = UInt(vpnLen.W)
}

val L2TlbMissQueueInDB, L2TlbMissQueueOutDB = Wire(new L2TlbMissQueueDB)
L2TlbMissQueueInDB.vpn := missQueue.io.in.bits.vpn
L2TlbMissQueueOutDB.vpn := missQueue.io.out.bits.vpn

val L2TlbMissQueueTable = ChiselDB.createTable(
    "L2TlbMissQueue_hart" + p(XSCoreParamsKey).HartId.toString, new L2TlbMissQueueDB)

L2TlbMissQueueTable.log(L2TlbMissQueueInDB, missQueue.io.in.fire, "L2TlbMissQueueIn", clock, reset)
L2TlbMissQueueTable.log(L2TlbMissQueueOutDB, missQueue.io.out.fire, "L2TlbMissQueueOut", clock, reset)

当你将缓存之间的总线事务记录到 SQLite3 数据库中时，你可以看到 ChiselDB 的结果，我们称之为 TL-Log，因为我们使用了 Tilelink 总线协议。它支持离线错误检测和性能分析。

让我们注入一个 bug 来演示如何使用 ChiselDB。我们提供了一个已编译的仿真器，该仿真器存在一个 bug，即从 L2 到 L3 缓存的所有释放数据都是常量值。

运行以下命令即可模拟预构建的仿真模型。新增了参数 -dump-db 来转储数据库。